In [1]:
import cv2 as cv
import numpy as np
import dlib
from pylibraries.centroidtracker import CentroidTracker
from pylibraries.trackableobject import TrackableObject
import paho.mqtt.client as mqtt

In [2]:
people_walking = "./video/walking.mp4"

In [3]:
topic = "teds20/group02/project" # Topic where messages are published
broker_address = "test.mosquitto.org" # Server where messages are published
client = mqtt.Client("P1")

In [4]:
confidence_threshold = 0.2 

CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]

net = cv.dnn.readNetFromCaffe('./mobilenet_ssd/MobileNetSSD.prototxt', './mobilenet_ssd/MobileNetSSD.caffemodel')

vs = cv.VideoCapture(people_walking)

centroid_tracker = CentroidTracker(maxDisappeared=40, maxDistance=50)
trackers = []
trackable_objects = {}

total_frames = 0
total_people = 0

try:
	client.connect(broker_address)
	client.loop_start()

	while vs.isOpened():
		(grabbed, frame) = vs.read()

		if not grabbed:
			break
		
		(h, w) = frame.shape[:2]
		scale_factor = 800 / w
		dim = (int(w * scale_factor), int(h * scale_factor))
		frame = cv.resize(frame, dim, interpolation = cv.INTER_AREA)
		RGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

		(h, w) = frame.shape[:2]

		rectangles = []

		if total_frames % 30 == 0:
			trackers = []

			blob = cv.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)

			net.setInput(blob)
			detections = net.forward()

			for i in np.arange(0, detections.shape[2]):
				confidence = detections[0, 0, i, 2]

				if confidence > confidence_threshold:
					idx = int(detections[0, 0, i, 1])

					if CLASSES[idx] != "person":
						continue

					box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
					(startX, startY, endX, endY) = box.astype("int")

					tracker = dlib.correlation_tracker()
					rectangle = dlib.rectangle(startX, startY, endX, endY)
					tracker.start_track(RGB, rectangle)

					trackers.append(tracker)
		else:
			for tracker in trackers:
				tracker.update(RGB)
				pos = tracker.get_position()

				startX = int(pos.left())
				startY = int(pos.top())
				endX = int(pos.right())
				endY = int(pos.bottom())

				rectangles.append((startX, startY, endX, endY))
		
		cv.line(frame, (0, h // 2), (w, h // 2), (255, 0, 255), 3)

		objects = centroid_tracker.update(rectangles)

		for (objectID, centroid) in objects.items():
			trackable_object = trackable_objects.get(objectID, None)

			if trackable_object is None:
				trackable_object = TrackableObject(objectID, centroid)
			else:
				y_coordinate = [c[1] for c in trackable_object.centroids]
				direction = centroid[1] - np.mean(y_coordinate)
				trackable_object.centroids.append(centroid)

				if not trackable_object.counted:
					if direction < 0 and centroid[1] < h // 2:
						total_people += 1
						client.publish(topic, "IN", 2)
						trackable_object.counted = True

					elif direction > 0 and centroid[1] > h // 2:
						total_people -= 1
						client.publish(topic, "OUT", 2)
						trackable_object.counted = True

			trackable_objects[objectID] = trackable_object

			id_text = "ID {}".format(objectID)
			cv.putText(frame, id_text, (centroid[0] - 10, centroid[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (50, 255, 50), 2)
			cv.circle(frame, (centroid[0], centroid[1]), 4, (50, 255, 50), -1)

		people_text = "{} people".format(total_people)
		cv.putText(frame, people_text, (10, h - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 150), 2)

		total_frames += 1

		cv.imshow("Frame", frame)
		key = cv.waitKey(1) & 0xFF

		if key == ord("q"):
			break

	client.publish(topic, "STOP", 2)
	client.loop_stop()
	client.disconnect()    
except Exception as e:
    print(f"connection error: {e}") 

vs.release()
cv.destroyAllWindows()